In [ ]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from transformers import BertTokenizer, BertModel, AdamW
import torchvision.models as models
from torch import nn
import time
from tqdm import tqdm

# Paths to the CSV files and image directories
csv_paths = {
    'train': '/kaggle/input/intent/Intent/train.csv',
    'test': '/kaggle/input/intent/Intent/validation.csv',
    'validation': '/kaggle/input/intent/Intent/validation.csv'
}

image_dirs = {
    'train': '/kaggle/input/intent/Intent/train',
    'test': '/kaggle/input/intent/Intent/validation',
    'validation': '/kaggle/input/intent/Intent/validation'
}
output_dir = '/kaggle/working/'  # Output directory to save the CSV files

# Function to check for matching Meme_ID and image files, and add image paths
def check_matches(csv_path, image_dir):
    df = pd.read_csv(csv_path)
    image_files = os.listdir(image_dir)
    image_names = {os.path.splitext(image_file)[0]: os.path.join(image_dir, image_file) for image_file in image_files}
    
    # Add Image_Path column to the dataframe
    df['Image_Path'] = df['Image_ID'].apply(lambda x: image_names.get(x, None))
    
    # Filter rows where Image_Path is not None (i.e., matched Meme_IDs)
    matched_df = df[df['Image_Path'].notna()]
    
    return matched_df

# Function to encode Intent_Taxonomy classes into labels
def encode_labels(df):
    label_encoder = LabelEncoder()
    df['Intent_Taxonomy_Labels'] = label_encoder.fit_transform(df['Intent_Taxonomy'])
    return df, label_encoder.classes_

# Check matches for each set (Train, Test, Validation)
for key in csv_paths:
    matched_df = check_matches(csv_paths[key], image_dirs[key])
    
    # Encode Intent_Taxonomy labels
    matched_df, classes = encode_labels(matched_df)
    
    matches_output_path = os.path.join(output_dir, f'{key}_matches.csv')
    
    # Save the processed dataframe to CSV
    matched_df.to_csv(matches_output_path, index=False)
    
    print(f"{key} set:")
    print(f"Matched Meme_IDs with image paths and labels saved to {matches_output_path}")
    print(f"Classes and their corresponding labels:\n{dict(zip(classes, range(len(classes))))}\n")

In [ ]:
train_df = pd.read_csv('/kaggle/working/train_matches.csv')
train_df.head(10)

In [ ]:
test_df = pd.read_csv('/kaggle/working/test_matches.csv')
test_df.head(10)

In [ ]:
validation_df = pd.read_csv('/kaggle/working/validation_matches.csv')
validation_df.head(10)

In [17]:
# Define your transformations using transforms.Compose
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),  # Crop the center to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class MyMultimodalDataset(Dataset):
    def __init__(self, image_paths, image_captions, intent_taxonomy_labels, transform=None):
        self.image_paths = image_paths
        self.image_captions = image_captions
        self.intent_taxonomy = intent_taxonomy_labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        text = self.image_captions[idx]
        label = self.intent_taxonomy[idx]

        # Load and preprocess image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, text, label

In [18]:
# Assuming you have lists or arrays of image paths, captions, and encoded labels:
train_dataset = MyMultimodalDataset(train_df['Image_Path'], train_df['Image_Caption'], train_df['Intent_Taxonomy_Labels'], transform=transform)
val_dataset = MyMultimodalDataset(validation_df['Image_Path'], validation_df['Image_Caption'], validation_df['Intent_Taxonomy_Labels'], transform=transform)
test_dataset = MyMultimodalDataset(test_df['Image_Path'], test_df['Image_Caption'], test_df['Intent_Taxonomy_Labels'], transform=transform)

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [113]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer,AdamW
from tqdm import tqdm
import torchvision.models as models
import time
from torchvision.models import mobilenet_v3_small

In [121]:
# Initialize mobilenet_v3_small with IMAGENET1K_V1 weights
mobilenet_v3_small = models.mobilenet_v3_small(weights='IMAGENET1K_V1', progress=True)
mobilenet_v3_small = torch.nn.Sequential(*(list(mobilenet_v3_small.children())[:-1]))  # Remove the classification layer
mobilenet_v3_model.to(device)

# Initialize BERT tokenizer and model
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
bert_model = BertModel.from_pretrained("bert-base-multilingual-cased")


In [ ]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
mobilenet_v3_small.to(device)

In [ ]:
bert_model.to(device)

In [126]:
import torch
import time
from torch.optim import AdamW
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import torch.nn as nn

In [ ]:
# Set models to evaluation mode
mobilenet_v3_small.eval()
bert_model.eval()

num_epochs = 35
num_classes = 6
max_seq_length = 100  # Set your desired maximum sequence length

train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

start_time = time.time()

# Define the Early Fusion Classifier
class EarlyFusionClassifier(nn.Module):
    def __init__(self, img_feature_dim, text_feature_dim, num_classes):
        super(EarlyFusionClassifier, self).__init__()
        self.fc1 = nn.Linear(img_feature_dim + text_feature_dim, 512)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, img_features, text_features):
        combined_features = torch.cat((img_features, text_features), dim=1)
        x = self.fc1(combined_features)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize Early Fusion classifier with corrected input dimensions
early_fusion_classifier = EarlyFusionClassifier(img_feature_dim=576, text_feature_dim=768, num_classes=6).to(device)

# Define optimizer and criterion
optimizer = AdamW(early_fusion_classifier.parameters(), lr=1e-5)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(num_epochs):
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, texts, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        # Move tensors to the device
        images = images.to(device)
        labels = labels.to(device)

        # Convert texts to tensors and pad to a fixed sequence length
        texts = [bert_tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt') for text in texts]
        input_ids = torch.stack([text['input_ids'].squeeze(0) for text in texts], dim=0).to(device)
        attention_mask = torch.stack([text['attention_mask'].squeeze(0) for text in texts], dim=0).to(device)

        optimizer.zero_grad()

        # Extract features from images and texts
        img_feats = mobilenet_v3_small(images)
        img_feats = img_feats.squeeze()
        text_outputs = bert_model(input_ids, attention_mask=attention_mask)
        text_feats = text_outputs.last_hidden_state[:, 0, :]

        # Ensure features are 2-dimensional
        if img_feats.dim() == 1:
            img_feats = img_feats.unsqueeze(0)
        if text_feats.dim() == 1:
            text_feats = text_feats.unsqueeze(0)


        # Forward pass through Early Fusion classifier
        combined_logits = early_fusion_classifier(img_feats, text_feats)

        # Ensure labels have the correct shape and type
        labels = labels.view(-1)  # Flatten labels to match batch size
        labels = labels.to(torch.long)  # Ensure labels are of type torch.long

        # Calculate loss
        loss = criterion(combined_logits, labels)

        # Backpropagation
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()

        # Calculate accuracy
        _, predicted = combined_logits.max(1)
        total_train += labels.size(0)
        correct_train += predicted.eq(labels).sum().item()

    epoch_train_loss = running_train_loss / len(train_loader)
    epoch_train_accuracy = correct_train / total_train

    train_losses.append(epoch_train_loss)
    train_accuracies.append(epoch_train_accuracy)

    # Validation loop
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for val_images, val_texts, val_labels in val_loader:
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_texts = [bert_tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt') for text in val_texts]
            val_input_ids = torch.stack([text['input_ids'].squeeze(0) for text in val_texts], dim=0).to(device)
            val_attention_mask = torch.stack([text['attention_mask'].squeeze(0) for text in val_texts], dim=0).to(device)

            # Extract features from images and texts for validation
            val_img_feats = mobilenet_v3_small(val_images)
            val_img_feats = val_img_feats.squeeze()
            val_text_outputs = bert_model(val_input_ids, attention_mask=val_attention_mask)
            val_text_feats = val_text_outputs.last_hidden_state[:, 0, :]

            # Ensure features are 2-dimensional
            if val_img_feats.dim() == 1:
                val_img_feats = val_img_feats.unsqueeze(0)
            if val_text_feats.dim() == 1:
                val_text_feats = val_text_feats.unsqueeze(0)

            # Forward pass through Early Fusion classifier for validation
            val_combined_logits = early_fusion_classifier(val_img_feats, val_text_feats)

            # Ensure val_labels have the correct shape and type
            val_labels = val_labels.view(-1)  # Flatten val_labels to match batch size
            val_labels = val_labels.to(torch.long)  # Ensure val_labels are of type torch.long

            # Calculate validation loss
            val_loss = criterion(val_combined_logits, val_labels)

            running_val_loss += val_loss.item()

            # Calculate validation accuracy
            _, val_predicted = val_combined_logits.max(1)
            total_val += val_labels.size(0)
            correct_val += val_predicted.eq(val_labels).sum().item()

    epoch_val_loss = running_val_loss / len(val_loader)
    epoch_val_accuracy = correct_val / total_val

    val_losses.append(epoch_val_loss)
    val_accuracies.append(epoch_val_accuracy)

    print(f"Epoch [{epoch + 1}/{num_epochs}] - "
          f"Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_accuracy:.4f}, "
          f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_accuracy:.4f}")

end_time = time.time()
execution_time = end_time - start_time
print(f"Total execution time: {execution_time:.2f} seconds")


In [ ]:
import torch
import torch.nn as nn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from tqdm import tqdm

# Assuming test_loader contains your test data loader

early_fusion_classifier.eval()  # Set the model to evaluation mode

test_losses = []
test_accuracies = []
predictions = []
true_labels = []

with torch.no_grad():
    for images, texts, labels in tqdm(test_loader, desc='Testing', leave=False):
        images = images.to(device)
        labels = labels.to(device)

        texts = [bert_tokenizer(text, padding='max_length', truncation=True, max_length=max_seq_length, return_tensors='pt') for text in texts]
        input_ids = torch.stack([text['input_ids'].squeeze(0) for text in texts], dim=0).to(device)
        attention_mask = torch.stack([text['attention_mask'].squeeze(0) for text in texts], dim=0).to(device)

        img_feats = mobilenet_v3_small(images)
        img_feats = img_feats.view(img_feats.size(0), -1)  # Flatten the image features

        outputs = bert_model(input_ids, attention_mask=attention_mask)
        text_feats = outputs.last_hidden_state[:, 0, :]  # Extract the [CLS] token representation

        if img_feats.dim() == 1:
            img_feats = img_feats.unsqueeze(0)
        if text_feats.dim() == 1:
            text_feats = text_feats.unsqueeze(0)

        logits = early_fusion_classifier(img_feats, text_feats)

        # Calculate loss if needed
        test_loss = criterion(logits, labels)
        test_losses.append(test_loss.item())

        # Calculate accuracy
        _, predicted = logits.max(1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())
        correct = predicted.eq(labels).sum().item()
        total = labels.size(0)
        test_accuracy = correct / total
        test_accuracies.append(test_accuracy)

# Calculate overall test metrics
average_test_loss = sum(test_losses) / len(test_losses)
test_accuracy = accuracy_score(true_labels, predictions)
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')
conf_matrix = confusion_matrix(true_labels, predictions)

# Print and plot results
print(f"Test Loss: {average_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1_score:.4f}")

In [ ]:
# Plot confusion matrix
plt.figure(figsize=(8, 6))
# Class names according to the label encoding mapping
class_names = ['Advocative', 'Controversial', 'ExhIbitionist', 'Expressive', 'Informative', 'Promotive']
sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='d', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()